In [15]:
import pandas as pd
import numpy as np
from progress.bar import Bar
from utils.load import load_data_methodology_cutting
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow.keras.backend as K
from utils.metrics import f1, confusion_matrix_plot
from utils.tuner import CVTuner
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import joblib
import keras_tuner
import tensorflow as tf
import os

In [2]:
df = pd.read_csv("../Metadata/data_curated.csv")
df.set_index("File", inplace=True)
df.head()

,Class
File,
S1V3C1M3A Clip166.npy,1
S1V1C1M1A Clip18.npy,1
S1V4C1M4A Clip241.npy,1
S1V4C1M4A Clip239.npy,1
S1V4C1M4A Clip245.npy,1


In [3]:
X_train, y_train, X_val, y_val = load_data_methodology_cutting(df,30,"M1")

In [4]:
X_train.shape

(2250, 30, 1662)

In [5]:
y_train.shape

(2250, 6)

In [6]:
X_val.shape

(750, 30, 1662)

In [7]:
y_val.shape

(750, 6)

# Cross Validation

In [8]:
cv = MultilabelStratifiedKFold(n_splits=3)
joblib.dump(cv, './cv/cv_GRU.joblib')

['./cv/cv_GRU.joblib']

# Hyper Parameter Tunning Recurrent Neural Networks

In [9]:
def build_model_GRU(hp):
    
    model = Sequential()


    model.add(LSTM(
        hp.Int('input_units',min_value=60,max_value=110,step=10),
        return_sequences=True,
        activation='relu', 
        kernel_initializer=tf.keras.initializers.GlorotNormal(),
        recurrent_initializer=tf.keras.initializers.Orthogonal(),
        recurrent_dropout=0.1,
        input_shape=(30,1662)))
    
    model.add(LSTM(
        hp.Int('GRU_1',min_value=110,max_value=160,step=10),
        return_sequences=True,
        activation='relu',
        recurrent_dropout=0.1,
        kernel_initializer=tf.keras.initializers.GlorotNormal(),
        recurrent_initializer=tf.keras.initializers.Orthogonal()
        ))
    

    model.add(LSTM(
        hp.Int('GRU_2',min_value=60,max_value=110,step=10), 
        recurrent_dropout=0.1,
        return_sequences=False,
        activation='relu',
        kernel_initializer=tf.keras.initializers.GlorotNormal(),
        recurrent_initializer=tf.keras.initializers.Orthogonal()
        ))
    

    model.add(Dense(
        hp.Int(f"Dense_units_1",min_value=60,max_value=110,step=10),
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(0.01)
        ))
    
    model.add(Dense(
        hp.Int(f"Dense_units_2",min_value=20,max_value=50,step=10),
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(0.01)
        ))
    

    model.add(Dense(6, activation='softmax'))

    opt = tf.keras.optimizers.Nadam(clipnorm = 1.0)
    model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=[f1])
    return model

In [10]:
tuner_LSTM = CVTuner(
                    data_cv= cv,
                    goal = 'f1',
                    hypermodel=build_model_GRU,
                    oracle=keras_tuner.oracles.BayesianOptimizationOracle(
                    objective=keras_tuner.Objective('f1',direction = "max"),
                        max_trials=1                  
                    ),
                    directory='./experiments/',
                    project_name = 'GRU',
                    overwrite=True
                    )

In [11]:
tuner_LSTM.search(X_train, y_train, 64, epochs=50)

Trial 1 Complete [00h 14m 03s]
f1: 0.395187646150589

Best f1 So Far: 0.395187646150589
Total elapsed time: 00h 14m 03s
INFO:tensorflow:Oracle triggered exit


In [12]:
df_LSTM = pd.DataFrame(tuner_LSTM.trial_scores)

In [13]:
df_LSTM.to_csv("./experiments/LSTM.csv")

In [33]:
def build_model_GRU():
    
    model = Sequential()


    model.add(LSTM(
        32,
        return_sequences=True,
        activation='relu', 
        kernel_initializer=tf.keras.initializers.GlorotNormal(),
        recurrent_initializer=tf.keras.initializers.Orthogonal(),
        recurrent_dropout=0.1,
        input_shape=(30,1662)))
    
    # model.add(LSTM(
    #     128,
    #     return_sequences=True,
    #     activation='relu',
    #     recurrent_dropout=0.1,
    #     kernel_initializer=tf.keras.initializers.GlorotNormal(),
    #     recurrent_initializer=tf.keras.initializers.Orthogonal()
    #     ))


    model.add(LSTM(
        16, 
        recurrent_dropout=0.1,
        return_sequences=False,
        activation='relu',
        kernel_initializer=tf.keras.initializers.GlorotNormal(),
        recurrent_initializer=tf.keras.initializers.Orthogonal()
        ))
    

    # model.add(Dense(
    #     64,
    #     activation='relu',
    #     kernel_regularizer=tf.keras.regularizers.l2(0.01)
    #     ))
    
    model.add(Dense(
        16,
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(0.01)
        ))
    

    model.add(Dense(6, activation='softmax'))



    opt = tf.keras.optimizers.Nadam(clipnorm = 1.0)
    model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=[f1])
    return model

In [34]:
log_dir = os.path.join("logs","experiments","Prueba" )
callback = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode = 'max',patience=10, verbose=1,restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1),
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
    tf.keras.callbacks.ModelCheckpoint(filepath=f'models/model_checkpoint/Prueba/0/best_model.h5', monitor="val_loss", save_best_only=True)
    ]

In [35]:
model = build_model_GRU()

In [37]:
model.fit(X_train, y_train, validation_data=(X_val,y_val),batch_size=64,epochs=50, callbacks=callback, verbose=1)

2024-01-08 22:23:20.543781: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 448740000 exceeds 10% of free system memory.


Epoch 1/50
36/36 [==============================] - 8s 223ms/step - loss: 3.1447 - f1: 0.4796 - val_loss: 1.7059 - val_f1: 0.1356 - lr: 1.2500e-04
Epoch 2/50


/home/andres/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


36/36 [==============================] - 7s 204ms/step - loss: 1.4154 - f1: 0.4934 - val_loss: 1.2219 - val_f1: 0.4342 - lr: 1.2500e-04
Epoch 3/50
36/36 [==============================] - 8s 228ms/step - loss: 1.1168 - f1: 0.5911 - val_loss: 1.0705 - val_f1: 0.4927 - lr: 1.2500e-04
Epoch 4/50
36/36 [==============================] - 8s 217ms/step - loss: 0.9442 - f1: 0.6587 - val_loss: 0.8911 - val_f1: 0.6306 - lr: 1.2500e-04
Epoch 5/50
36/36 [==============================] - 8s 233ms/step - loss: 0.8311 - f1: 0.7074 - val_loss: 0.9467 - val_f1: 0.6300 - lr: 1.2500e-04
Epoch 6/50
36/36 [==============================] - 9s 241ms/step - loss: 0.7103 - f1: 0.7605 - val_loss: 0.8012 - val_f1: 0.7822 - lr: 1.2500e-04
Epoch 7/50
36/36 [==============================] - 8s 229ms/step - loss: 0.6537 - f1: 0.7876 - val_loss: 0.7375 - val_f1: 0.7981 - lr: 1.2500e-04
Epoch 8/50
36/36 [==============================] - 8s 212ms/step - loss: 0.6017 - f1: 0.8187 - val_loss: 0.8233 - val_f1: 0.7651

In [14]:
#disp = confusion_matrix_plot(y_val, y_pred)
#disp.plot()